## Import libraries

In [ ]:
import os

import copick
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go

%matplotlib widget

## Load data

In [ ]:
copick_filepath = "/home/michele/DNN/kaggle/kaggle_cryo_et/dataset/train/copick.config"
numpy_anns_sphere_dir = "/home/michele/DNN/kaggle/kaggle_cryo_et/dataset/train/numpy_sphere"
numpy_anns_point_dir = "/home/michele/DNN/kaggle/kaggle_cryo_et/dataset/train/numpy_point"
experiments_dir = "/home/michele/DNN/kaggle/kaggle_cryo_et/dataset/train/static/ExperimentRuns"
voxel_spacing = 10
tomo_type = "denoised"
available_runs = os.listdir(experiments_dir)
print(f"Available runs: {available_runs}")
root_copick = copick.from_file(copick_filepath)

### Select data

In [ ]:
experiment_to_show = "TS_5_4"
annotation_type = "sphere"

experiment_ann_sphere = np.load(os.path.join(numpy_anns_sphere_dir, experiment_to_show + ".npy"))
experiment_ann_points = np.load(os.path.join(numpy_anns_point_dir, experiment_to_show + ".npy"))

## Visualize data

Draw the data points in the quantized voxel space

In [ ]:
def draw_3d_labels_plotly(root_copick, experiment_ann, plotly_fig, marker_size=3):
    for obj in root_copick.pickable_objects:
        if not obj.is_particle:
            continue
        print(f"Name: {obj.name}, radius: {obj.radius}, color: {obj.color}")
        # Set color corresponding to label
        particle_type_bool = experiment_ann == obj.label
        # Get tuple of z, y, x values
        label_volume_coordinates = particle_type_bool.nonzero()
        z_values = label_volume_coordinates[0]
        y_values = label_volume_coordinates[1]
        x_values = label_volume_coordinates[2]
        print(f"Number of volume points to draw: {len(z_values)}")
        plotly_fig.add_trace(
            go.Scatter3d(
                x=x_values,
                y=y_values,
                z=z_values,
                mode="markers",
                name=obj.name,
                marker=dict(
                    color=f"rgb({obj.color[0]}, {obj.color[1]}, {obj.color[2]})",
                    size=marker_size
                )
            ),
        )

def draw_copick_points_plotly(root_copick, plotly_fig, marker_size=3):
    for obj in root_copick.pickable_objects:
        if not obj.is_particle:
            continue
        print(f"Name: {obj.name}, radius: {obj.radius}, color: {obj.color}")
        picks = run.get_picks(obj.name, user_id="curation", session_id="0")
        assert len(picks) == 1
        # Visualize each annotation independently
        x_values = []
        y_values = []
        z_values = []
        for point in picks[0].points:
            # Center
            x_values.append(round(point.location.x / voxel_spacing))
            y_values.append(round(point.location.y / voxel_spacing))
            z_values.append(round(point.location.z / voxel_spacing))
        print(f"Number of copick points to draw: {len(z_values)}")
        plotly_fig.add_trace(
            go.Scatter3d(
                x=x_values,
                y=y_values,
                z=z_values,
                mode="markers",
                name=obj.name,
                marker=dict(
                    color=f"rgb({obj.color[0]}, {obj.color[1]}, {obj.color[2]})",
                    size=marker_size
                )
            )
        )

def update_plotly_layout(plotly_fig, max_x, max_y, max_z, axis_range_ext, title):
    fig.update_layout(
    title=dict(
        text=title,
        font=dict(size=20)
    ),
    legend=dict(
        font=dict(size=12)
    ),
    scene = dict(
        xaxis = dict(range=[0, round(volume_shape[2] * 1.1)], zerolinecolor="black", gridcolor="gray"),
        yaxis = dict(range=[0, round(volume_shape[1] * 1.1)], zerolinecolor="black", gridcolor="gray"),
        zaxis = dict(range=[0, round(volume_shape[0] * 1.1)], zerolinecolor="black", gridcolor="gray"),
    ),
    autosize=False,
    width=800,
    height=800,
    paper_bgcolor="LightSteelBlue",
)

### Draw sphere volumes from processed numpy

WARNING: plotly is way faster than matplotlib, but visualizing the sphere of points could still be slow and occupy a lot of memory

The volume to draw has already the sphere around the target 3D point. The radius of the sphere depend on the particle type

In [ ]:
volume_shape = experiment_ann_sphere.shape
print(f"Volume shape: {volume_shape}")

fig = go.Figure()
draw_3d_labels_plotly(root_copick, experiment_ann_sphere, fig, marker_size=3)
update_plotly_layout(
    fig,
    max_x=volume_shape[2],
    max_y=volume_shape[1],
    max_z=volume_shape[0],
    axis_range_ext=1.1,
    title=f"Numpy 3D spheres {experiment_to_show}"
)
print("Building figure to show...")
fig.show()

### Draw points from processed numpy

Draw the target 3D points in the quantized voxel space

In [ ]:
volume_shape = experiment_ann_sphere.shape
print(f"Volume shape: {volume_shape}")

fig = go.Figure()
draw_3d_labels_plotly(root_copick, experiment_ann_points, fig, marker_size=3)
update_plotly_layout(
    fig,
    max_x=volume_shape[2],
    max_y=volume_shape[1],
    max_z=volume_shape[0],
    axis_range_ext=1.1,
    title=f"Numpy 3D points {experiment_to_show}"
)
print("Building figure to show...")
fig.show()

### Draw centroids from copick format

Use the original copick format to draw the 3D points, this should be equivalent to the numpy points annotations

In [ ]:
fig = go.Figure()
draw_3d_labels_plotly(root_copick, experiment_ann_points, fig, marker_size=3)
update_plotly_layout(
    fig,
    max_x=volume_shape[2],
    max_y=volume_shape[1],
    max_z=volume_shape[0],
    axis_range_ext=1.1,
    title=f"Copick centroids {experiment_to_show}"
)
print("Building figure to show...")
fig.show()

matplotlib fallback for reference, the number of points is manageable with matplotlib as well, although it is more laggy

In [ ]:
ax = plt.figure(f"Copick centroids {experiment_to_show}").add_subplot(projection='3d')
xs = []
ys = []
zs = []
colors = []
for obj in root_copick.pickable_objects:
    if not obj.is_particle:
        continue
    print(f"Name: {obj.name}, radius: {obj.radius}, color: {obj.color}")
    picks = run.get_picks(obj.name, user_id="curation", session_id="0")
    assert len(picks) == 1
    # Visualize each annotation independently
    for point in picks[0].points:
        # Center
        xs.append(round(point.location.x / voxel_spacing))
        ys.append(round(point.location.y / voxel_spacing))
        zs.append(round(point.location.z / voxel_spacing))
        colors.append(np.array(obj.color[:3]) / 255)
ax.scatter(xs, ys, zs, c=colors)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.show()